In [188]:
import pandas as pd
import numpy as np

from geopandas import GeoDataFrame
import shapely
from shapely import wkt
from shapely.geometry import Polygon, Point

import matplotlib.pyplot as plt

In [189]:
buyouts = pd.read_csv('Buyout_agreements.csv')
buyouts = buyouts.drop(columns=['DELETE - Neighborhoods',
       'DELETE - Police Districts', 'DELETE - Supervisor Districts',
       'DELETE - Fire Prevention Districts', 'DELETE - Zip Codes',])
buyouts = buyouts[4:]

In [190]:
buyouts['point'] = buyouts['point'].dropna().apply(wkt.loads)
# buyouts

In [191]:
sf = pd.read_csv('sf polygons.csv')
sf = sf.set_index(sf.name.apply(lambda x: x.replace('a','')))
sf['geometry']=sf['geometry'].apply(wkt.loads)
sf['name'] = sf.name.apply(lambda x: x.replace('a',''))

In [192]:
res=[]
for point in buyouts['point']:
    if type(point)!=shapely.geometry.point.Point:
        res.append(0)
    else:
        count=0
        for poly in sf['geometry']:
            if poly.contains(point):
                res.append(poly)
                count+=1
        if count<1:
            res.append(0)    

In [193]:
sf['geo_str']=sf['geometry'].astype(str)

In [194]:
buyouts['poly']=res
buyouts['geo_str'] = buyouts['poly'].astype(str)
# buyouts.merge(sf,left_on='poly',right_on='geometry')

In [195]:
merged = buyouts.merge(sf[['geo_str','name']],left_on='geo_str',right_on='geo_str')
merged.to_csv('buyouts.csv')

In [198]:
buyouts['point']

4       POINT (-122.4212366775404 37.79896367700326)
5       POINT (-122.4271065003205 37.77255335488871)
6       POINT (-122.4227368302494 37.76423321969258)
7       POINT (-122.4247068768847 37.79107476095793)
8        POINT (-122.499420903674 37.77402355983759)
                            ...                     
4368    POINT (-122.4235044132087 37.76801173002097)
4369    POINT (-122.4442530562239 37.79028364260326)
4370    POINT (-122.4963748075345 37.77628682767589)
4371     POINT (-122.441007180837 37.77167167277432)
4372     POINT (-122.433223286518 37.79648397212929)
Name: point, Length: 4369, dtype: object

In [196]:
buyouts['point'].apply(lambda z: z.x)
buyouts['point'].apply(lambda z: z.y)

AttributeError: 'float' object has no attribute 'x'

In [187]:
buyouts['point'].iloc[0].x

-122.42123667754045